In [1]:
import numpy as np
import tensorflow as tf

from keras.models import Model
from keras.layers import Dense,  Flatten

from keras.preprocessing import image
from tensorflow.keras.preprocessing import image

from keras.applications import VGG16

from PIL import Image
import os

from keras.models import load_model, save_model

from keras.callbacks import EarlyStopping

from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K

from sklearn.preprocessing import LabelBinarizer

import matplotlib.pyplot as plt

In [ ]:
'''
Manual preprocessing for image datasets
'''

image_folder = r"../img"
images = []
labels = []
i = 0

for folder_imgName in os.listdir(image_folder):

    for files_name in os.listdir(image_folder + '/' + folder_imgName):

        if files_name.endswith(".png"):
            folder_path = image_folder + '/' + folder_imgName

            # img loading
            img = Image.open(os.path.join(folder_path, files_name))
            lbl = folder_imgName

            # preprocessing image
            img = img.resize((224, 224))  # resizingf for model
            img = np.array(img) / 255.0  # Normalisation (between 0 & 1)


            images.append(img)
            labels.append(lbl)

            if i == 0:
                img_test= img
                i==1


# preprocessing labels & convert to numpy array
labelB = LabelBinarizer()
train_labels = labelB.fit_transform(labels)

# image list to numpy array
train_images = np.array(images)



In [ ]:
'''
Manual preprocessing for image datasets
'''


image_folder = r"../img_val"
images = []
labels = []
i = 0

for folder_imgName in os.listdir(image_folder):

    for files_name in os.listdir(image_folder + '/' + folder_imgName):

        if files_name.endswith(".png"):
            folder_path = image_folder + '/' + folder_imgName

            # img loading
            img = Image.open(os.path.join(folder_path, files_name))
            lbl = folder_imgName

            # preprocessing image
            img = img.resize((224, 224))  # resizingf for model
            img = np.array(img) / 255.0  # Normalisation (between 0 & 1)


            images.append(img)
            labels.append(lbl)

            if i == 0:
                imgs_test= img
                i==1


# preprocessing labels & convert to numpy array
labelB = LabelBinarizer()
val_labels = labelB.fit_transform(labels)

# image list to numpy array
val_images = np.array(images)



In [ ]:
# définition des paramètres personalisés

# personal layers for VGG16 models
n_classes = 7
n_layers = 7
n_neurons_BeforeLast = 128
f_activation = 'relu'
f_activation_lastLayer = 'softmax'

# personal parameters for compilation
optimizer = 'Adagrad'
loss = 'categorical_crossentropy'

# personnal parameters for training (fitness)
epochs = 100
batch_size = 32

#numéro de l'essai
num_essai = '0_VGG16'

In [ ]:
# Model VGG16 loading without fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freeze convolution layers to avoid training
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add personnal layers for training
x = Flatten()(base_model.output)

for i in range(n_layers):
    x = Dense(n_neurons_BeforeLast * (n_classes-i), activation=f_activation)(x) #personal layer

predictions = Dense(n_classes, activation=f_activation_lastLayer)(x)  # last layer of model for output

In [ ]:
# generate model VGG16 with personal fully connected layers
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
# def scheduler(epoch, lr):
#     if epoch < 12:
#         return lr
#     else:
#         return lr * K.exp(-0.1)

# lr_scheduler = LearningRateScheduler(scheduler)


In [ ]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [ ]:
# # Model training vgg16
# model.fit(train_ds, epochs= epochs, batch_size= batch_size, validation_data=validation_ds)


# Model training
model.fit(train_images, train_labels, epochs= epochs, batch_size= batch_size, validation_data=(val_images, val_labels), callbacks=[early_stopping])#, lr_scheduler

Epoch 1/100
175/175 [==============================] - 24s 117ms/step - loss: 1.5903 - accuracy: 0.4502 - val_loss: 1.3726 - val_accuracy: 0.3795
Epoch 2/100
175/175 [==============================] - 20s 115ms/step - loss: 1.0085 - accuracy: 0.6275 - val_loss: 0.8801 - val_accuracy: 0.6304
Epoch 3/100
175/175 [==============================] - 20s 115ms/step - loss: 0.7668 - accuracy: 0.7079 - val_loss: 0.6257 - val_accuracy: 0.7884
Epoch 4/100
175/175 [==============================] - 20s 115ms/step - loss: 0.6295 - accuracy: 0.7630 - val_loss: 0.5939 - val_accuracy: 0.7768
Epoch 5/100
175/175 [==============================] - 20s 114ms/step - loss: 0.5561 - accuracy: 0.7871 - val_loss: 0.4833 - val_accuracy: 0.8134
Epoch 6/100
175/175 [==============================] - 20s 114ms/step - loss: 0.4951 - accuracy: 0.8111 - val_loss: 0.4925 - val_accuracy: 0.7955
Epoch 7/100
175/175 [==============================] - 20s 114ms/step - loss: 0.4614 - accuracy: 0.8261 - val_loss: 0.4097 -

In [ ]:
model.save(f"../model_saved/{num_essai}_model.h5py")

INFO:tensorflow:Assets written to: ../model_saved/0_VGG16_model.h5py\assets


INFO:tensorflow:Assets written to: ../model_saved/0_VGG16_model.h5py\assets
